[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ivanvladimir/maquinas_notebooks/blob/main/lfya/03%20Las%20m%C3%A1quina%20que%20est%C3%A1n%20en%20varios%20lugares.ipynb)

## 00 Instalando la librería 

Se instala la librería [maquinas](https://pypi.org/project/maquinas/)

**Requerido en colab**, _opcional en ambiente local a través de jupyter_

In [ ]:
# Execute if not installed
!pip install maquinas

### 01 Importar módulo para cargar autómatas finitos y el módulo de simiulación

Existen dos elemento: _load_fa_, _las reducciones_, el proceso de _minimización_, _RE_ y _Simulation_ (en este momento _Simulación_ sólo puede ir hacia adelante)

In [ ]:
from maquinas.io import load_fa
from maquinas.regular.ndfa_e import NonDeterministicFiniteAutomaton_epsilon as NDFA_e
from maquinas.regular.dfa import DeterministicFiniteAutomaton as DFA
from maquinas.regular.reductions import *
from maquinas.regular.RE import RegularExpression as RE
from maquinas.regular.minimization import *
from maquinas.simulation import Simulation

## 02 Automatas Finitos No Determinísticos

Se puede crear un [AFND](https://ivanvladimir.gitlab.io/lfya_book/docs/03maquinasqueest%C3%A1nenvarioslugares/02automatafinitonodeterministico/#aut%c3%b3mata-finito-no-determin%c3%adstico) desde la definición tipo tabla de uno de estos; la función _load_fa_ infiere que se trata de un _AFND_, y no un _AF_, porque alguna celda tiene más de un estado.

In [ ]:
chicles_1=load_fa("""
             |               1               |               2               |               5               |
 ->q₀        | q₁₁/₁,q₁₁/₅,q₁₁/₃,q₁₁/₄,q₁₁/₂ |       q₁₂/₇,q₁₂/₈,q₁₂/₆       |              q₅               |
   q₁₁/₁     |             q₂₁/₁             |                               |                               |
   q₂₁/₁     |             q₃₁/₁             |                               |                               |
   q₃₁/₁     |             q₄₁/₁             |                               |                               |
   q₄₁/₁     |              q₅               |                               |                               |
   q₁₁/₂     |                               |           q₁₁_₁₂/₂            |                               |
   q₁₁_₁₂/₂  |                               |              q₅               |                               |
   q₁₁/₃     |                               |           q₁₁_₁₂/₃            |                               |
   q₁₁_₁₂/₃  |           q₂₁_₁₂/₃            |                               |                               |
   q₂₁_₁₂/₃  |              q₅               |                               |                               |
   q₁₁/₄     |             q₂₁/₄             |                               |                               |
   q₂₁/₄     |                               |           q₂₁_₁₂/₄            |                               |
   q₂₁_₁₂/₄  |              q₅               |                               |                               |
   q₁₁/₅     |             q₂₁/₅             |                               |                               |
   q₂₁/₅     |             q₃₁/₅             |                               |                               |
   q₃₁/₅     |                               |              q₅               |                               |
   q₁₂/₆     |                               |             q₂₂/₆             |                               |
   q₂₂/₆     |              q₅               |                               |                               |
   q₁₂/₇     |           q₁₁_₁₂/₇            |                               |                               |
   q₁₁_₁₂/₇  |                               |              q₅               |                               |
   q₁₂/₈     |           q₁₁_₁₂/₈            |                               |                               |
   q₁₁_₁₂/₈  |           q₂₁_₁₂/₈            |                               |                               |
   q₂₁_₁₂/₈  |              q₅               |                               |                               |
   q₅]       |                               |                               |                               |
""")
chicles_1.graph()

In [ ]:
print(chicles_1.summary())

### 02.a Tabla de un AFND

Es posible imprimir la tabla de una máquina (estado inicial marcado con "⟶" y finales con color verde)

In [ ]:
chicles_1.table()

### 02.b  Transitando un AFND

La aplicación de [delta extendida](https://ivanvladimir.gitlab.io/lfya_book/docs/03maquinasqueest%C3%A1nenvarioslugares/02automatafinitonodeterministico/#funci%c3%b3n-de-transici%c3%b3n-extendida-para-afnds) paso a paso

In [ ]:
for q,a,w_ in chicles_1.delta_stepwise("122"):
    if a:
        print(f"{a} -> {q}", end=",\n ")
    else:
        print(f"{q}",end="\n ")
    res=q
    
print(f"\nCon {res} Se acepta?", "Sí" if chicles_1.acceptor(res) else "No" )

### 02.c Simulación de AFND

In [ ]:
s=Simulation(chicles_1,"122")
s.run()

## 03 Reducción de AFND → AFD

Para reducir un [AFND a AFD](https://ivanvladimir.gitlab.io/lfya_book/docs/03maquinasqueest%C3%A1nenvarioslugares/03afnd_af/), utilizamos la función _ndfa2dfa_; notar que se usa el parámetro _rename_ en _False_ para ver la codificación utilizada

In [ ]:
chicles_2_full=ndfa2dfa(chicles_1,rename=False)
chicles_2_full.print_summary()

In [ ]:
chicles_2_full.table()

In [ ]:
chicles_2_full.graph()

### 03.a Redución renombrando estados

In [ ]:
chicles_2=ndfa2dfa(chicles_1)
chicles_2.table()

In [ ]:
chicles_2.graph()

In [ ]:
for q,a,w_ in chicles_2.delta_stepwise("122"):
    if a:
        print(f"{a} -> {q}", end=",\n ")
    else:
        print(f"{q}",end="\n ")
    res=q
    
print(f"\nCon {res} Se acepta?", "Sí" if chicles_2.acceptor(res) else "No" )

## 04 AFND-ε

En este caso, en lugar de usar la función _load_fa_, usamos directametne la clase _NDFA_e_ para agregar la transición epsilón que permite el cobro múltiple de pagos.

In [ ]:
chicles_3=NDFA_e(Q=chicles_2.Q,
                 sigma=chicles_2.sigma,
                 q_0=chicles_2.q_0,
                 A=chicles_2.A,
                 delta=[(k,qs) for k,qs in chicles_2.items()])
for q_f in chicles_3.A:
    chicles_3.add_transition(q_f,'ε',['q_0'])

In [ ]:
chicles_3.graph()

In [ ]:
chicles_3.table()

### 04.a  Transitando un AFND-ε

La aplicación de [delta extendida](https://ivanvladimir.gitlab.io/lfya_book/docs/03maquinasqueest%C3%A1nenvarioslugares/04automatafinitonodeterministico_epsilon/#funci%c3%b3n-de-transici%c3%b3n-extendida-para-afnd-%ce%b5) paso a paso

In [ ]:
for q,a,w_ in chicles_3.delta_stepwise("5212"):
    if a:
        print(f"{a} -> {q}", end=",\n ")
    else:
        print(f"{q}",end="\n ")
    res=q
    
print(f"\nCon {res} Se acepta?", "Sí" if chicles_3.acceptor(res) else "No" )

In [ ]:
s=Simulation(chicles_3,"5212")
s.run()

## 05 Reducción de AFND-ε → AFND

Para reducir un [AFND-ε → AFND](https://ivanvladimir.gitlab.io/lfya_book/docs/03maquinasqueest%C3%A1nenvarioslugares/05afnd_e_afnd/#reducci%c3%b3n), utilizamos la función _ndfa_e2ndfa_; notar que se usa el parámetro _rename_ en _False_ para ver la codificación utilizada

In [ ]:
chicles_4=ndfa_e2ndfa(chicles_3, rename=False)
chicles_4.graph()

In [ ]:
chicles_4.table()

### 05.a Reduciendo la reducción

El _AFND_ es posible reducirlo a _AFD_

In [ ]:
chicles_5=ndfa2dfa(chicles_4)
chicles_5.graph()

### 05.b Agregando el pago 2,2,2,2,2

Para agregar un pago que permita el pago de [_2,2,2,2,2_](https://ivanvladimir.gitlab.io/lfya_book/docs/03maquinasqueestánenvarioslugares/05afnd_e_afnd/#pago_22222) se tiene que agregar una transición;

In [ ]:
# agregando aceptar el pago: 2,2,2,2,2
chicles_5.add_transition('q_7','2',['q_1'],force=True)
chicles_5.graph()

In [ ]:
for q,a,w_ in chicles_5.delta_stepwise("22222"):
    if a:
        print(f"{a} -> {q}", end=",\n ")
    else:
        print(f"{q}",end="\n ")
    res=q
    
print(f"\nCon {res} Se acepta?", "Sí" if chicles_3.acceptor(res) else "No" )

## 06. Operation en AF

Es posible aplicar las [operaciones de lenguajes regulares](https://ivanvladimir.gitlab.io/lfya_book/docs/03maquinasqueest%C3%A1nenvarioslugares/07_afnd_e_er/#aplicando-operaciones-de-lenguajes-regulares-a-afnd-%ce%b5) a autómatas finitos; observar que antes de aplicar la operación es necesario convertir el [AFD a AFND-ε](https://ivanvladimir.gitlab.io/lfya_book/docs/03maquinasqueest%C3%A1nenvarioslugares/06af_afnd_e/)

In [ ]:
# Creating automata finito

ceros_uno=load_fa("""
       | 0  | 1  |
-> q0  | q0 | q1 |
   q1] |    |    |

""")
ceros_uno=dfa2ndfa_e(ceros_uno) 
ceros_uno.graph()

In [ ]:
uno_ceros=load_fa("""
       | 0  | 1  |
-> q0  |    | q1 |
   q1] | q1 |    |

""")
uno_ceros=dfa2ndfa_e(uno_ceros)
uno_ceros.graph()

### 06.a Unión

In [ ]:
union=ceros_uno.union(uno_ceros)
union.save_img('union')
union.graph()

### 06.b Concatenación

In [ ]:
concat=ceros_uno.concat(uno_ceros)
concat.save_img('concat')
concat.graph()

### 06.c Cerradura estrella

In [ ]:
kleene=ceros_uno.kleene()
kleene.save_img("cerradura")
kleene.graph()

## 07 ER → AFND-ε

Es posible producir una [ER a un AFND-ε](https://ivanvladimir.gitlab.io/lfya_book/docs/03maquinasqueest%C3%A1nenvarioslugares/07_afnd_e_er/#ejemplo-de-afnd-%ce%b5-a-er)

In [ ]:
reg = RE('(a*ba*ba*)*+a*')
reg=reg.ndfa_e()

In [ ]:
reg.graph()

In [ ]:
reg_a = RE('a')
reg_a=reg_a.ndfa_e()
reg_a.save_img("reg_a")
reg_a.graph()

In [ ]:
reg_a = RE('a*')
reg_a=reg_a.ndfa_e()
reg_a.save_img("reg_a_klene")
reg_a.graph()

In [ ]:
reg_a_b = RE('a*b')
reg_a_b=reg_a_b.ndfa_e()
reg_a_b.save_img("reg_a_b_klene")
reg_a_b.graph()

## 08 Minimización

La librería también implementa un proceso de minimización

In [ ]:
# Creating automata finito
one_one=DFA(Q=['A','B','C','D','E','F'],
                         sigma=['0','1'],
                         q_0='A',
                         A=['C','D','E'],
                         delta=[
                            (('A','0'),'B'),
                            (('A','1'),'C'),
                            (('B','0'),'A'),
                            (('B','1'),'D'),
                            (('C','0'),'E'),
                            (('C','1'),'F'),
                            (('D','0'),'E'),
                            (('D','1'),'F'),
                            (('E','0'),'E'),
                            (('E','1'),'F'), 
                            (('F','0'),'F'),
                            (('F','1'),'F'), 
                         ])

In [ ]:
one_one.graph()

In [ ]:
one_one.print_summary()

In [ ]:
one_one_min=minimization_hopcroft(one_one,rename=False)

In [ ]:
one_one_min.graph()

In [ ]:
# Creating automata finito
bes_divisible_by_3=DFA(Q=['q_0','q_1','q_2','q_3'],
                         sigma=['a','b'],
                         q_0='q_0',
                         A=['q_0','q_3'],
                         delta=[
                            (('q_0','a'),['q_0']),
                            (('q_0','b'),['q_1']),
                            (('q_1','a'),['q_1']),
                            (('q_1','b'),['q_2']),
                            (('q_2','a'),['q_2']),
                            (('q_2','b'),['q_3']),
                            (('q_3','a'),['q_3']),
                            (('q_3','b'),['q_1']),
                         ])
bes_divisible_by_3.graph()

In [ ]:
mini=minimization_hopcroft(bes_divisible_by_3)
mini.graph()